# 미션 1: 야놀자 리뷰 요약

## 🎯 학습 목표

이 미션에서는 야놀자 숙박 리뷰 데이터를 활용하여 리뷰를 요약하는 AI 서비스를 개발합니다.

### 주요 학습 내용
- LLM을 활용한 텍스트 요약
- 프롬프트 엔지니어링 기초
- 리뷰 데이터 전처리 및 분석

## 📝 실습 내용

### 구현 기능
- 야놀자 리뷰 데이터 수집/로드
- 리뷰 데이터 전처리
- LLM을 활용한 리뷰 요약
- 요약 결과 평가


### LLM 모델
- gpt-3.5-turbo

## 라이브러리 로드

In [1]:
import json
import sys
import time
import os
from tqdm import tqdm

# Crawling
from bs4 import BeautifulSoup
from selenium import webdriver

# 날짜 관련 데이터 전처리
import datetime
from dateutil import parser

# LLM
from dotenv import load_dotenv
from openai import OpenAI

# 데모
import gradio as gr

/home/user/Desktop/Camp/ai-camp-holiday-mission-JAE-HUN-CHO/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 야놀자 리뷰 데이터 수집/로드

In [ ]:
# 야놀자 리뷰 Crawling 함수
def crawl_yanolja_reviews(name, url):
    os.makedirs('./res', exist_ok=True)
    
    review_list = []
    driver = webdriver.Chrome()
    driver.get(url)

    time.sleep(3)

    scroll_count = 20
    for i in range(scroll_count):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(2)

    # Crawling
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    review_containers = soup.select('#__next > section > div > div.css-1js0bc8 > div > div > div')
    review_date = soup.select('#__next > section > div > div.css-1js0bc8 > div > div > div > div.css-1toaz2b > div > div.css-1ivchjf')

    # Dictionary 형태로 저장
    for i in range(len(review_containers)):
        review_text = review_containers[i].find('p', class_='content-text').text
        review_stars = review_containers[i].select('path[fill="currentColor"]')
        star_cnt = sum(1 for star in review_stars if not star.has_attr('fill-rule'))
        date = review_date[i].text

        review_dict = {
            'review': review_text,
            'stars': star_cnt,
            'date': date
        }

        review_list.append(review_dict)

    with open(f'./res/{name}.json', 'w') as f:
        json.dump(review_list, f, indent=4, ensure_ascii=False)


In [3]:
# Crawling
review_name_url_dict_list = [
    ## 나인트리 인사동
    {"name" : 'reviews', 'url' : 'https://nol.yanolja.com/reviews/domestic/1000102261?sort=HOST_CHOICE'},
    ## 나인트리 판교
    {"name" : 'ninetree_pangyo', 'url' : 'https://nol.yanolja.com/reviews/domestic/1000113873?sort=HOST_CHOICE'},
    ## 나인트리 용산
    {"name" : 'ninetree_yongsan', 'url' : 'https://nol.yanolja.com/reviews/domestic/10048873?sort=HOST_CHOICE'},
]

for review_name_url_dict in review_name_url_dict_list:
    crawl_yanolja_reviews(
        name=review_name_url_dict['name'], 
        url=review_name_url_dict['url']
    )

In [4]:
with open('./res/reviews.json', 'r') as f:
    review_list = json.load(f)

review_list[:10]

[{'review': '걸어디니는 거리에 볼수있는곳이 많아 좋았어요\n깨끗하고 공기도 쾌적하고 조계사뷰도 좋았습니다\n서울에서 대중교통이용하며 다니기 편한 숙소입니다',
  'stars': 5,
  'date': '4일 전'},
 {'review': '에어컨 사용이 쎄서 그런가 건조한거 빼고는 전반적으로 서비스 좋았습니다',
  'stars': 4,
  'date': '2025.09.25'},
 {'review': '깔끔하고 위치가 좋아요', 'stars': 5, 'date': '2025.09.22'},
 {'review': '조식도 맛있고 직원분들도 친절하시고 만족합니다.', 'stars': 5, 'date': '2025.09.21'},
 {'review': '깨끗하고 친절하고 좋습니다', 'stars': 5, 'date': '2025.09.08'},
 {'review': '좋았습니다좋았습니다', 'stars': 5, 'date': '2025.09.05'},
 {'review': '깔끔하고 위치 좋음', 'stars': 5, 'date': '2025.09.05'},
 {'review': '방과 침구는 깨끗하고 조명도 포근해서 좋았어요', 'stars': 5, 'date': '2025.09.03'},
 {'review': '좋습니다. 주차비가 비싼건 아쉽습니다.', 'stars': 4, 'date': '2025.09.02'},
 {'review': '잘 이용했습니다~~', 'stars': 5, 'date': '2025.08.30'}]

## 리뷰 데이터 전처리

In [5]:
# 리뷰 데이터 전처리
def preprocess_reviews(path='./res/reviews.json'):
    # Json 파일 로드
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    # 6 개월 이전 데이터 필터링
    current_date = datetime.datetime.now()
    data_boundary = current_date - datetime.timedelta(days=30*6)
    filtered_cnt = 0

    # Crawling된 리뷰 데이터 기반 전처리
    for r in review_list: 
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        # 날짜 필터링
        if review_date < data_boundary:
            continue
        
        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)
    return reviews_good_text, reviews_bad_text


In [6]:
reviews, _ = preprocess_reviews()
print(reviews)

[REVIEW_START]걸어디니는 거리에 볼수있는곳이 많아 좋았어요
깨끗하고 공기도 쾌적하고 조계사뷰도 좋았습니다
서울에서 대중교통이용하며 다니기 편한 숙소입니다[REVIEW_END]
[REVIEW_START]깔끔하고 위치가 좋아요[REVIEW_END]
[REVIEW_START]조식도 맛있고 직원분들도 친절하시고 만족합니다.[REVIEW_END]
[REVIEW_START]깨끗하고 친절하고 좋습니다[REVIEW_END]
[REVIEW_START]좋았습니다좋았습니다[REVIEW_END]
[REVIEW_START]깔끔하고 위치 좋음[REVIEW_END]
[REVIEW_START]방과 침구는 깨끗하고 조명도 포근해서 좋았어요[REVIEW_END]
[REVIEW_START]잘 이용했습니다~~[REVIEW_END]
[REVIEW_START]역시실망시키질않네요 ㅎㅎ깨끗하고 궁근처에서는 가성비와 시설 모든면이좋은듯합니다[REVIEW_END]
[REVIEW_START]잘 쉬다갑니다 좋아요[REVIEW_END]
[REVIEW_START]그냥 최고입니다
층수가 높으면 조계사가 잘 보였을텐데 아쉬움이 남더군요[REVIEW_END]
[REVIEW_START]아주 좋았습니다! 뷰는 도시뷰라 어쩔수 없지만..청와대 경복궁 조계사가 보여요![REVIEW_END]
[REVIEW_START]위치도 너무 좋고 시설이 깔끔해서 좋았어요~
우선 침대랑 샤워실,화장실이 유리로 되어있지 않고 따로 구분되어 있어서 좋았어요~
저희가 더위를 잘타는데 에어컨도 빵빵해서 중간에 더워서 일어날일 없이 푹 자서 좋았어요😆
다음에도 인사동 올일 있다면 다시 오고 싶은 곳이에요~[REVIEW_END]
[REVIEW_START]깔끔하고 방음이 좋습니다. 채광도 좋았습니다!
다시 방문할 의향이 있습니다.[REVIEW_END]
[REVIEW_START]좋아요 야경도 좋고 접근성도 좋았어요[REVIEW_END]
[REVIEW_START]위치 최고, 깨끗한 시설, 친절한 서비스, 약간의 방음 아쉬움[REVIEW_END]


## LLM을 활용한 리뷰 요약

In [ ]:
# LLM 환경 변수 및 OpenAI 로드
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI(
    api_key=OPENAI_API_KEY,
)

In [9]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5 문장 내로 요약해줘"""

In [10]:
# 리뷰 요약
def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', "content" : prompt}],
        temperature=temperature
    )
    return completion

In [11]:
print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)

다양한 관광지와 가까운 거리에 위치한 깨끗하고 쾌적한 숙소로, 대중교통 이용이 편리하며 조계사 뷰도 좋았다. 위치가 좋고 조식도 맛있으며 직원들도 친절하고 만족스러웠다. 방과 침구가 깨끗하고 조명이 포근하며, 숙소는 깔끔하고 위치가 좋았다. 숙소는 깨끗하고 친절하며, 뷰가 좋아 재방문 의사가 100%이다.


## 요약 결과 평가

In [12]:
# 리뷰 데이터 기반 평가 함수
## 실제 요약 및 생성된 요약 비교
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt =f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": eval_prompt}
        ],
        temperature=0.1
    )

    return completion

In [13]:
# 평가 함수를 여러 번 반복하여 성능 평가
def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i])
        verdict_text = completion.choices[0].message.content

        if '[[A]]' in verdict_text:
            a_cnt += 1
        elif '[[B]]' in verdict_text:
            b_cnt += 1
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        else:
            print(f'Error: {verdict_text}')
    return a_cnt, b_cnt, draw_cnt

In [14]:
result_dict = {
    'baseline' : {"wins" : 0, "losses" : 0, 'ties' : 0},
    'improving_prompt' : {"wins" : 0, "losses" : 0, 'ties' : 0},
    'improving_data' : {"wins" : 0, "losses" : 0, 'ties' : 0},
    'one_shot' : {"wins" : 0, "losses" : 0, 'ties' : 0},
    'two_shot' : {"wins" : 0, "losses" : 0, 'ties' : 0},
}

### Prompt baseline

In [15]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5 문장 내로 요약해줘"""

In [16]:
summary_real = """숙소는 서울 인사동 인근에 위치해 있어 접근성이 뛰어나며, 주변 관광지와의 연계성이 우수합니다. 청결 상태가 양호하고 방음이 비교적 잘 되어 있어 편안한 휴식을 제공합니다. 직원들은 친절하며, 조식과 어메니티도 만족스럽다는 평가가 많습니다. 조각된 뷰와 쾌적한 환경이 장점으로 언급되며, 가족 단위 방문객에게도 적합한 공간으로 보입니다. 전반적으로 시설과 서비스에 대한 긍정적인 후기가 많습니다."""

In [17]:
eval_count = 10
summaries = [summarize(reviews, PROMPT_BASELINE, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}') 

100%|██████████| 10/10 [00:18<00:00,  1.90s/it]

Wins: 7, Losses: 3, Ties: 0


In [18]:
result_dict['baseline']['wins'] = wins
result_dict['baseline']['losses'] = losses
result_dict['baseline']['ties'] = ties

### Prompt 고도화

In [19]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

In [20]:
eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

 80%|████████  | 8/10 [00:15<00:04,  2.02s/it]

Error: Assistant A's response provides a detailed summary of the user reviews, mentioning the convenient location, cleanliness, pleasant atmosphere, good views, friendly staff, and overall satisfaction of the guests. The response also highlights the positive aspects of the accommodation, such as the good breakfast, cleanliness, and quiet environment, indicating a high likelihood of repeat visits. On the other hand, Assistant B's response also covers similar points, emphasizing the excellent accessibility, cleanliness, good soundproofing, friendly staff, satisfactory breakfast, and amenities. Both responses accurately capture the key points from the user reviews and provide a comprehensive overview of the accommodations.

However, Assistant A's response stands out for its slightly more detailed and creative language, mentioning specific attractions like 조계사 (Jogyesa Temple) and using phrases like "쾌적한 환경" (pleasant environment) and "재방문 의사" (intention to revisit). Assistant A's response

100%|██████████| 10/10 [00:19<00:00,  1.91s/it]

Wins: 5, Losses: 4, Ties: 0


In [21]:
result_dict['improving_prompt']['wins'] = wins
result_dict['improving_prompt']['losses'] = losses
result_dict['improving_prompt']['ties'] = ties

### 입력 데이터의 품질 증가

In [ ]:
# 리뷰 데이터 전처리
## 글자 수 필터링 추가
def preprocess_reviews(path='./res/reviews.json'):
    # Json 파일 로드
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    # 6 개월 이전 데이터 필터링
    current_date = datetime.datetime.now()
    data_boundary = current_date - datetime.timedelta(days=30*6)
    filtered_cnt = 0

    # Crawling된 리뷰 데이터 기반 전처리
    for r in review_list: 
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        # 날짜 필터링
        if review_date < data_boundary:
            continue
        
        # 글자 수 필터링
        if len(r['review']) < 30:
            filtered_cnt += 1
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)
    return reviews_good_text, reviews_bad_text


In [23]:
reviews, _ = preprocess_reviews()

In [24]:
eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:18<00:00,  1.85s/it]

Wins: 9, Losses: 1, Ties: 0


In [25]:
result_dict['improving_data']['wins'] = wins
result_dict['improving_data']['losses'] = losses
result_dict['improving_data']['ties'] = ties

### Few-Shot Prompting

#### One-Shot

In [27]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_1shot, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:20<00:00,  2.03s/it]

Wins: 7, Losses: 3, Ties: 0


In [28]:
result_dict['one_shot']['wins'] = wins
result_dict['one_shot']['losses'] = losses
result_dict['one_shot']['ties'] = ties

#### Two-Shot

In [29]:
reviews_1shot, _ = preprocess_reviews('./res/ninetree_pangyo.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0).choices[0].message.content
reviews_2shot, _ = preprocess_reviews('./res/ninetree_yongsan.json')
summary_2shot = summarize(reviews_2shot, prompt_1shot, temperature=0.0).choices[0].message.content

prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.

예시 리뷰들 1:
{reviews_1shot}
예시 요약 결과 1:
{summary_1shot}

예시 리뷰들 2:
{reviews_2shot}
예시 요약 결과 2:
{summary_2shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_2shot, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:18<00:00,  1.88s/it]

Wins: 3, Losses: 7, Ties: 0


In [30]:
result_dict['two_shot']['wins'] = wins
result_dict['two_shot']['losses'] = losses
result_dict['two_shot']['ties'] = ties

## 결과 시각화

In [31]:
import pandas as pd

df = pd.DataFrame.from_dict(result_dict, orient='index')
print(df)

                  wins  losses  ties
baseline             7       3     0
improving_prompt     5       4     0
improving_data       9       1     0
one_shot             7       3     0
two_shot             3       7     0
